## CNN Convolutional Neural Network pour Classification des Images

Pour cette deuxième partie du tutorial nous allons Réaliser un Réseau de Convolution mais cette fois-ci sur un jeu de données plus complexe que les chiffres du MNIST. En effet nous allons utiliser un dataset avec des **vrais images pour la reconnaissance Chien / Chat** (toujours en apprentissage supervisé)

Nous Utiliserons encore une fois les libraries de KERAS pour la création du réseau de Convolution.

Tous les principaux éléments qui composent ce réseau ont été abordés en partie 1, s'y reporter en cas de besoin.

### Image Classification Références:
https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6
https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
https://www.kaggle.com/stevenhurwitt/cats-vs-dogs-using-a-keras-convnet
https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
https://github.com/shervinea/enzynet/blob/master/scripts/architecture/enzynet_uniform.py
https://stanford.edu/~shervine/blog/evolution-image-classification-explained
https://www.analyticsvidhya.com/blog/2016/10/tutorial-optimizing-neural-networks-using-keras-with-image-recognition-case-study/
http://ruder.io/optimizing-gradient-descent/
https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

dataset : https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data


## Import des librairies nécessaire

Ci-dessous nous importons toutes les libraries nécessaires pour la création du réseau de la même manière que dans le précédent tutorial.

Nous allons également rajouter des librairies de Keras pour la gestion des callback (nous allons voir ce point plus en détail)

In [ ]:
import os
import pandas as pd
import numpy as np

from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.utils import class_weight as cw
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Flatten, Dense, Dropout

# Import des librairies pour la gestion des Callback
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping

## Configuration du modèle

Afin de variabiliser et de configurer plus facilement le réseau nous exposons ici les principaux paramètres configurables

In [ ]:
EPOCHS                  = 100   # Nombre d'epoch
IMGSIZE                 = 96    # Taille des images
BATCH_SIZE              = 32    # Pour le traitement par lot des images (optimisation de la decente de gradient)
STOPPING_PATIENCE       = 10    # Callback pour stopper si le modèle n'apprend plus
VERBOSE                 = 0     # Niveau de verbosité
MODEL_NAME              = 'cnn_80epochs_imgsize160'
OPTIMIZER               = 'adam'
TRAINING_DIR            = '../input/dogs-vs-cats-redux-kernels-edition/train'
TEST_DIR                = '../input/dogs-vs-cats-redux-kernels-edition/test'
TRAIN_MODEL             = True  # Entrainement du modele (True) ou chargement (False)

## Préparation des Jeux de données

Création du dataframe des **données d'entrainement** contenant les id des images et leur labels.
Nous utilisons toujours le Dataframe de Panda pour construire nos tableaux de données.

Cette fois ci le jeu d'entrainement contient le nom du fichier ainsi que le label de l'image dans le nom du fichier (ie cat.100.jpg)

Nous allons donc créer un tableau avec la colonne id contenant le nom du fichier et la colonne label contenant la classe à prédire.

In [ ]:
train_files = os.listdir(TRAINING_DIR)
train_labels = []

for file in train_files:
    train_labels.append(file.split(".")[0])
    
df_train = pd.DataFrame({"id": train_files, "label": train_labels})

df_train.head()

## Data Generators & Image Real Time Augmentation

### Jeu d'entrainement 
**Augmentation des images** à la volée via les générateurs pour permettre de simuler une augmentation du nombre de données disponible pour l'entrainement du réseau. 

Comme vu précédement c'est la classe de **Keras ImageDataGenerator** qui va permettre l'application de filtres et de transformations sur les images sources.

Il faut également noter ici que nous faisons **split du jeu d'entrainement en deux sous parties** (les données d'entrainement et celles de validation)
Ceci est réalisé grâce à validation_split de ImageDataGenerator pour lequel on préciser un ratio.

In [ ]:
# Augmentation d'images à la volée et split train / validation
train_datagen =  \
        ImageDataGenerator(
            rescale=1./255,
            shear_range=0.1,
            zoom_range=0.3,
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True,
            vertical_flip=True,
            validation_split=0.10)

# Parcours du jeu d'entrainement (subset = 'training')
train_generator = \
        train_datagen.flow_from_dataframe(
            df_train,
            TRAINING_DIR,
            x_col='id',
            y_col='label',
            has_ext=True,
            shuffle=True,
            target_size=(IMGSIZE, IMGSIZE),
            batch_size=BATCH_SIZE,
            subset='training',
            class_mode='categorical')

### Jeu de Validation
Traitement du jeu de **validation qui est une sous partie du jeu d'entrainement** (subset = 'validation'). 

A noter que pour le jeu de validation nous appliquons aussi l'augmentation des images à la volée (utilisation du train_datagen) , ce n'est pas une obligation.

In [ ]:
valid_generator = \
        train_datagen.flow_from_dataframe(
            df_train,
            TRAINING_DIR,
            x_col='id',
            y_col='label',
            has_ext=True,
            shuffle=True,
            target_size=(IMGSIZE, IMGSIZE),
            batch_size=BATCH_SIZE,
            subset='validation',
            class_mode='categorical')

### Jeu de Test
Le jeu de test contient les images non classifiées sur lesquels nous devons faire les prédictions pour déterminer si l'image est un chien ou un chat.

Nous allons donc suivre le même processus que pour les données de l'entrainement sauf qu'ici : 
* Nous ne connaissons pas les labels des images (evidence)
* Nous n'appliquons pas de transformations sur les images

In [ ]:
test_files = os.listdir(TEST_DIR)
df_test = pd.DataFrame({"id": test_files, 'label': 'nan'})

In [ ]:
# https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
# Le ImageDataGenerator fait juste une normalisation des valeurs
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_dataframe(
    df_test, 
    TEST_DIR, 
    x_col='id',
    y_col=None,       # None car nous ne connaissons pas les labels
    has_ext=True, 
    target_size=(IMGSIZE, IMGSIZE), 
    class_mode=None,  # None pour le jeu de test
    seed=42,
    batch_size=1,     # batch_size = 1 sur le jeu de test
    shuffle=False     # Pas de mélange sur le jeu de test
)

In [ ]:
# Cette fonction permet de retourner le ratio entre chat vs chien (utile dans le cas ou une classe et proéminente sur les autres)
def get_weight(y):
    class_weight_current =  cw.compute_class_weight('balanced', np.unique(y), y)
    return class_weight_current
class_weights = get_weight(train_generator.classes)

Avec l'utilisation des generator il est nécessaire de maitriser les "step_size" : 

In [ ]:
# Génération des STEPS_SIZE (comme nous utilisons des générateurs infinis)
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size
STEP_SIZE_TEST  = test_generator.n  // test_generator.batch_size

## Callbacks Keras

Petite nouveauté par rapport à la partie 1 : **L'utilisation des Callbacks**.

Les callback permettent d'appliquer des traitements pendant l'entrainement du réseau.  Nous pouvons donc influer ou observer l'apprentissage en cours.

Dans notre cas nous allons en définir deux : un pour permettre **l'arrêt prématuré** de l'entrainement afin d'économiser les temps de calcul dans le cas ou le réseau ne progresse plus, et le deuxième pour influer sur un paramètre appelé le **Learning Rate** utilisé dans les calculs numériques de la decente de gradient.

In [ ]:
# Permet de stopper l'apprentissage si il stagne
EARLY_STOPPING = \
        EarlyStopping(
            monitor='val_loss',
            patience=STOPPING_PATIENCE,
            verbose=VERBOSE,
            mode='auto')


# Reduit le LearningRate si stagnation
LR_REDUCTION = \
        ReduceLROnPlateau(
            monitor='val_acc',
            patience=5,
            verbose=VERBOSE,
            factor=0.5,
            min_lr=0.00001)

CALLBACKS = [EARLY_STOPPING, LR_REDUCTION]

## Architecture du CNN

Ci-dessous comme nous l'avons déjà vu nous définissons l'architecture du CNN puis de sa couche de classification (cf partie 1).

Comme nous ne prédisons que deux classes, **la couche de sortie sera composée de deux neurones**.

In [ ]:
# Initialisation du modèle
classifier = Sequential()

# Réalisation des couches de Convolution  / Pooling

# ---- Conv / Pool N°1
classifier.add(Conv2D(filters=16,
                      kernel_size=3,
                      strides=1,
                      padding='same',
                      input_shape=(IMGSIZE, IMGSIZE, 3),
                      activation='relu'))

classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# ---- Conv / Pool N°2
classifier.add(Conv2D(filters=16,
                      kernel_size=3,
                      strides=1,
                      padding='same',
                      activation='relu'))

classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# ---- Conv / Pool N°3
classifier.add(Conv2D(filters=32,
                      kernel_size=3,
                      strides=1,
                      padding='same',
                      activation='relu'))

classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# ---- Conv / Pool N°4
classifier.add(Conv2D(filters=32,
                      kernel_size=3,
                      strides=1,
                      padding='same',
                      activation='relu'))

classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2, 2), strides=2))


# Fully Connected
# Flattening : passage de matrices 3D vers un vecteur
classifier.add(Flatten())
classifier.add(Dense(512, activation='relu'))
classifier.add(Dropout(0.1))


# Couche de sortie : classification => softmax sur le nombre de classe
classifier.add(
    Dense(
        units=2,
        activation='softmax',
        name='softmax'))

# compilation du  model de classification
classifier.compile(
    optimizer=OPTIMIZER,
    loss='categorical_crossentropy',
    metrics=['accuracy'])


print("Input Shape :{}".format(classifier.get_input_shape_at(0)))
classifier.summary()

## Entrainement du modèle

Il est temps d'entrainer le modèle que nous avons crée avec les données que nous avons préparées.

In [ ]:
def train_model():
    # https://keras.io/models/sequential/#fit_generator
    # Pour visualisation avec Tensorboard (console anaconda): 
    # tensorboard --logdir=/full_path_to_your_logs
    history = classifier.fit_generator(
        generator=train_generator,           # le générateur pour les données d'entrainement
        steps_per_epoch=STEP_SIZE_TRAIN,     # le Step_size pour les données d'entrainement
        validation_data=valid_generator,     # le générateur pour les données de validation
        validation_steps=STEP_SIZE_VALID,    # le Step_size pour les données de validation
        epochs=EPOCHS,                       # le nombre d'epoch sur l'ensemble du jeu de données
        verbose=VERBOSE,                     # la verbosité
        class_weight=class_weights,          # le ratio de répartition des classes chien/chat
        callbacks=CALLBACKS)                 # la liste des fonctions de callback à appeler après chaque epoch
    return history    

In [ ]:
def plot_history(history):
    # --------------------------------------
    # Affichage des courbes accuracy et Loss
    # --------------------------------------
    plt.figure(1)
    plt.subplot(211)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')

    plt.subplot(212)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()     

### Entrainement ou Chargement du modèle

**Cette étape permet de charger un modèle déjà entrainé**.

Pour charger le modèle il faut d'abord avoir entrainé le réseau, commité le notebook et ensuite uploadé en zip le fichier contenant les poids et l'architecture du modèle. Il est également nécessaire de configurer la variable TRAIN_MODEL = False.

A la suite de l'entrainement nous pouvons observer les courbes d'accuracy et de loss pour vérifier si le modèle apprend correctement

In [ ]:
if (TRAIN_MODEL):
    print("Entrainement du modèle CNN")
    hist = train_model()     # Entrainement du modèle
    plot_history(hist)       # Affichage de la courbe d'apprentissage
    classifier.save(MODEL_NAME + '.h5')
else:
    print("Chargement du modèle...")
    classifier = load_model('../input/weight/cnn/cnn.h5')

## Evaluations du modèle

Nous procédons maintenant à l'évaluation de la performance du modèle sur le jeu de Validation. 
La première valeur est le Loss et la seconde l'accuracy

In [ ]:
classifier.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_TEST)

## Prédictions

### Génération des prédictions depuis le modèle

Le modèle étant entrainé il est temps de générer les prédictions sur les images, nous obtenons en sortie deux probabilités fournies par la couche de sortie du modèle de classification.

In [ ]:
# Le générateur doit être reseter avant utilisation pour les prédictions
test_generator.reset()
pred=classifier.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=1)

In [ ]:
# Visualisation du vecteur de probabilité des 5 premières lignes des prédictions
pred[0:5,:]

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
# Création d'un dataframe contenant les images et classes prédites
filenames=test_generator.filenames
results=pd.DataFrame({"id":filenames,"label":predictions})
results.head()

### Mise en forme des prédictions
Mise en forme pour préparer le format attendu pour la soumission des résultats

In [ ]:
# copy du dataframe de resultat
soumission = results.copy()

# suppression de l'extension du fichier et conversion de la colonne en int avec la méthode vectorielle str
soumission['id'] = soumission['id'].str[:-4].astype('int')
soumission.head()

In [ ]:
# Tri sur la colonne des id avec la methode sort_values du dataframe
soumission = soumission.sort_values(by=['id'])
soumission.head()

In [ ]:
# Remplacement du label 'cat' ou 'dog' par une valeur numérique : utilisation de la fonction replace
# Rappel sur les classes : {0: "Cat", 1: "Dog"} 
soumission.replace({'dog': 1, 'cat': 0}, inplace=True)
soumission.head()

### Ecriture du fichier de soumission

In [ ]:
# conversion du Dataframe vers un fichier de sortie
# This is saved in the same directory as your notebook
filename = 'results.csv'
soumission.to_csv(filename,index=False)
print('Fichier enregistré: ' + filename)

## Affichage aléatoire des images prédites

In [ ]:
import random

n = results.shape[0]
f = list(np.arange(1,n))

c = 20
r =random.sample(f, c)
nrows = 4
ncols = 5
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(nrows*5, ncols*5))    
for i in range(c):
    file = str(results['id'][r[i]])
    path = TEST_DIR+"/"+file
    img = plt.imread(path)
    plt.subplot(4, 5, i+1)
    plt.imshow(img, aspect='auto')
    plt.xticks([])
    plt.yticks([])
    plt.title(str(results['id'][r[i]])+"\n"+str(results['label'][r[i]]))
plt.show()